# DAMASK WORKFLOWS
This jupyter notebook describes a typical DAMASK workflow. There can be variations of this workflow, but this is the simplest possible test case.

The DAMASK version used here is - development version (version number: )

Currently, run this jupyter notebook on maws03 workstation



## Pre-processing 
### Geometry generation
Create a geometry file for DAMASK and also (possibly?) visualise it


### Load definition